### purpose

set up code to run the complex sims through RONA scripts

In [1]:
from pythonimports import *

import MVP_01_train_gradient_forests as mvp01
import MVP_05_train_RONA as mvp05
import MVP_summary_functions as mvp

lview, dview = get_client(cluster_id='1677852350-uvth', profile='lotterhos')

t1 = dt.now()  # notebook timer

outerdir = '/home/b.lind/offsets/run_20220919_tutorial'
tdir = '/home/b.lind/offsets/run_20220919_tutorial/tutorial'

training_dir = op.join(outerdir, 'gradient_forests/training/training_files')
gf_outdir = training_dir.replace('/training_files', '/training_outfiles')
gf_shdir = training_dir.replace('/training_files', '/training_shfiles')
rona_training_dir = training_dir.replace('/gradient_forests/', '/RONA/')
rona_outdir = makedir(rona_training_dir.replace('/training_files', '/training_outfiles'))

mvp.latest_commit()
session_info.show()

36 36
#########################################################
Today:	March 03, 2023 - 12:50:04
python version: 3.8.5

Current commit of pythonimports:
commit 8d4f83fd91caa2e58acbafcadfde07d19ae79379  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Fri Mar 3 00:48:36 2023 -0500

Current commit of MVP_offsets:
commit d24363688641c3c7e26b5202d74aefc5db4de83c  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Wed Mar 1 16:34:25 2023 -0500
#########################################################



In [2]:
# get data for the individuals that were subsampled from full simulation
subset = mvp01.read_ind_data(mvp.slimdir, '1231094')

subset.head()


Reading in info for subsampled individuals ...


,seed,subpopID,indID,indSubpopIndex,subpop,phen_sal,phen_temp,sal_opt,temp_opt,fitness,subset,N,opt0,opt1,x,y,PC1,PC2,PC3,LFMM_U1_temp,LFMM_U1_sal,LFMM_U2_temp,LFMM_U2_sal,RDA1,RDA2,RDA1_corr,RDA2_corr,RDA_PC1,RDA_PC2,RDA_PC1_corr,RDA_PC2_corr,RDA_predict_tempPhen_20KSNPs,RDA_predict_salPhen_20KSNPs,RDA_predict_tempPhen_20KSNPs_structcorr,RDA_predict_salPhen_20KSNPs_structcorr,sample_name
0,1231094,1,0,0,1,0,-0.918613,-1.0,-1.0,0.986840,True,10,-1.0,-1.0,1,1,78.2873,53.5668,-35.4246,30.981432,54.625443,-16.291995,33.276982,-2.493963,-4.524617,-5.509492,-4.531535,-3.065134,-2.694464,-0.595289,4.598294,-1.463450,-2.243004,-2.105670,-2.085053,0
1,1231094,1,1,1,1,0,-1.121660,-1.0,-1.0,0.970833,True,10,-1.0,-1.0,1,1,81.2725,53.6842,-32.7762,31.640989,57.098593,-14.176016,34.060294,-2.604211,-2.623063,-5.097704,-2.618050,-3.126949,-2.429470,-0.270668,4.483603,-1.523478,-1.380582,-1.986905,-1.207887,1
2,1231094,1,2,2,1,0,-1.066450,-1.0,-1.0,0.991208,True,10,-1.0,-1.0,1,1,76.9650,53.2122,-33.7540,31.949163,54.278986,-15.251630,34.242743,-2.476464,-4.242721,-5.004671,-4.235992,-3.155337,-2.564676,-0.570673,4.129895,-1.445380,-2.135772,-1.946000,-1.977806,2
3,1231094,1,3,3,1,0,-1.028410,-1.0,-1.0,0.998387,True,10,-1.0,-1.0,1,1,77.0485,51.2987,-33.5278,29.571425,53.819297,-14.588372,31.841537,-2.455641,-3.917719,-5.136301,-3.921065,-2.933899,-2.481845,-0.663457,3.841083,-1.427528,-1.967423,-1.973979,-1.812017,3
4,1231094,1,4,4,1,0,-1.024840,-1.0,-1.0,0.998767,True,10,-1.0,-1.0,1,1,81.5754,53.9046,-31.0938,31.850800,57.598035,-12.993029,34.303806,-2.624548,-2.664925,-4.592443,-2.647879,-3.146499,-2.282039,-0.135839,4.194792,-1.532787,-1.361768,-1.773886,-1.187342,4


In [3]:
mvp05.get_envdata(subset)  # dview['popenvdata']


Getting environmental data for each subpopID ...


,sal_opt,temp_opt
1,-1.000000,-1.0
2,-0.777778,-1.0
3,-0.555556,-1.0
4,-0.333333,-1.0
5,-0.111111,-1.0
...,...,...
96,0.111111,1.0
97,0.333333,1.0
98,0.555556,1.0
99,0.777778,1.0


# main

### step 1

In [4]:
envdata = pd.read_table(op.join(training_dir, 'tutorial_envfile_GFready_pooled.txt'), index_col=0)

envdata

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
1,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
2,0.070353,-0.265649,0.799727,0.169512,-0.417965,0.315295
3,0.264633,-0.248372,0.375248,-0.172690,-0.586408,-0.320803
4,0.371616,0.167485,0.063118,-0.465978,-0.656053,-0.625240
5,0.244613,0.117437,-0.010939,-0.291486,-0.528748,-0.537376
...,...,...,...,...,...,...
96,-0.288220,0.261489,-0.317123,-0.018548,0.228030,-0.045559
97,-0.116570,0.323553,-0.231351,-0.269762,0.344912,0.059459
98,0.093915,0.400397,-0.150975,-0.408936,0.588880,0.274592
99,0.034177,0.399089,-0.440848,-0.464035,0.673533,0.360201


In [5]:
# # get data for the individuals that were subsampled from full simulation
# subset = read_ind_data(slimdir, seed)

# # get environmental values for each subpopID, load to parallel engines
# dview['popenvdata'] = get_envdata(subset)  # load popenvdata to engines

dview['popenvdata'] = envdata

### step 2

In [6]:
freqfiles = {}

for f in fs(rona_training_dir, pattern='-gt-'):
    marker_set = op.basename(f).split("_")[-1].rstrip('.txt')
    
    print(marker_set)
    
    freqfiles[marker_set] = f

adaptive
all
neutral


In [7]:
# mvp05.calculate_linear_models:

# # calculate linear model fits between pop allele freq and climate env
# results, freqfile = calculate_linear_models(seed)
# dview['results'] = results  # load results to engines
# sleeping(10)

from MVP_05_train_RONA import train
train.__module__ = '__main__'  # so that train can see dview

# calculate linear models in parallel
jobs = parallel_read(freqfiles['all'],
                     lview=lview,
                     dview=dview,
                     functions=create_fundict(train),
                     verbose=False,
                     maintain_dataframe=False,
                     index_col=0)

# gather results
results = defaultdict(dict)
for envdict in jobs:
    for env, locusdict in envdict.items():
        results[env].update(locusdict)


respkl = op.join(rona_outdir, f'tutorial_linear_model_results.pkl')
pkldump(results, respkl)

respkl


Watching 36 parallel_read() jobs ...


tutorial_Rout_Gmat_sample_maf-gt-p01_RONAready_pooled_all.txt: 100%|███████████████| 36/36 [00:04<00:00,  7.61


Function `parallel_read` completed after : 0-00:00:13


'/home/b.lind/offsets/run_20220919_tutorial/RONA/training/training_outfiles/tutorial_linear_model_results.pkl'

In [21]:
from MVP_05_train_RONA import get_rona_elements, calc_rona_elements
get_rona_elements.__module__ = '__main__'
calc_rona_elements.__module__ = '__main__'

In [24]:
sig_models = mvp05.retrieve_significant_models(results)

dview['results'] = results  # load results to engines

dview['sig_models'] = sig_models  # load sig_models to engines

# calculate the locus-level element of the RONA equation
dview['calc_rona_elements'] = calc_rona_elements  # load function to engines

sleeping(10)  # make sure sig_models loads completely to all engines


Determining which loci had significant linear models ...


env6_PWM: 100%|███████████████| 26371/26371 [00:00<00:00, 1834416.72it/s]


env1_mat had 10181 loci with significant linear models out of 26371 loci
env2_MTWetQ had 13058 loci with significant linear models out of 26371 loci
env3_MTDQ had 8516 loci with significant linear models out of 26371 loci
env4_PDM had 12316 loci with significant linear models out of 26371 loci
env5_PwarmQ had 10018 loci with significant linear models out of 26371 loci
env6_PWM had 8461 loci with significant linear models out of 26371 loci


In [25]:
# mvp05.scatter_rona_elements:

# def calc_rona_elements(garden, env, pop, aaf_present):
#     """Calculate and return the abs element for each of n loci in `aaf_present.index`."""
#     ef_fut = popenvdata[env].loc[garden]
    
#     rona_elements = {}
#     for locus_i in aaf_present.index:
#         s_present, i_present, _pval = results[env][locus_i]
#         # nans can be appended if locus has missing data for that pop, account for this when summing/avging
#         rona_elements[locus_i] = abs(
#             (s_present * ef_fut) + i_present - aaf_present[pop].loc[locus_i]
#         )
    
#     return rona_elements

# def get_rona_elements(freqs):
#     """Calculate the typical summation element of RONA for loci with significant linear models."""
#     from pythonimports import wrap_defaultdict
#     from tqdm import tqdm as pbar
    
#     freqs.columns = freqs.columns.astype(int)  # convert from string to int to avoid pop name lookup errors
    
#     elements = wrap_defaultdict(None, 3)
#     for env,loci in sig_models.items():
#         interloci = set(loci).intersection(freqs.index)  # don't want whole table - need because freqs is a chunk
#         for garden in pbar(popenvdata.index, desc=env):
#             for pop in freqs.columns:
#                 elements[env][garden][pop] = calc_rona_elements(garden,
#                                                                 env,
#                                                                 pop,
#                                                                 freqs.loc[interloci])
#     return elements

print(
    ColorText(
        '\nCalculating locus elements of RONA for loci with significant linear models ...'
    ).bold().custom('gold')
)

# calculate rona elements in parallel
jobs = parallel_read(freqfiles['all'],
                     lview=lview,
                     dview=dview,
                     verbose=False,
                     index_col=0,
                     functions=create_fundict(get_rona_elements),
                     maintain_dataframe=False)

# gather elements
rona_elements = wrap_defaultdict(dict, 3)
for j in pbar(jobs, desc='extracting element data'):
    for env,gardendict in j.items():
        for garden,popdict in gardendict.items():
            for pop,elementdict in popdict.items():
                # `elementdict` is returned from `calc_rona_elements` - key=locus, val=summation_element
                rona_elements[env][garden][pop].update(elementdict)
                


Calculating locus elements of RONA for loci with significant linear models ...

Watching 36 parallel_read() jobs ...


tutorial_Rout_Gmat_sample_maf-gt-p01_RONAready_pooled_all.txt: 100%|███████████████| 36/36 [08:15<00:00, 13.77


Function `parallel_read` completed after : 0-00:08:23


extracting element data: 100%|███████████████| 36/36 [01:21<00:00,  2.27s/it]


In [43]:
marker_loci = {}
for marker_set, f in freqfiles.items():
    df = pd.read_table(f, index_col=0)
    
    marker_loci[marker_set] = df.index.tolist()
    
    print(marker_set, nrow(df))
    
df.head()

adaptive 90
all 26371
neutral 13285


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
11-500069,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.388889,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.227273,0.30,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0
11-500093,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.136364,0.1875,0.078947,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.111111,0.37500,0.272727,0.0,0.0,0.0,0.0,0.0,0.0,0.15,0.153846,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.611111,0.0,0.00,0.0,0.0,0.0,0.0
11-500129,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.5,0.363636,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.285714,0.0625,0.000000,0.6875,0.078947,0.0,0.0,0.0,0.0,0.0,0.033333,0.066667,0.222222,0.59375,0.454545,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.692308,0.7,0.272727,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.055556,0.0,0.05,0.0,0.0,0.0,0.0
11-500130,0.0,0.0,0.0,0.0,0.0,0.0,0.722222,0.500000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.272727,0.25,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0
11-500132,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.1,0.111111,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.7,0.071429,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.346154,0.055556,0.0,0.000000,0.0000,0.000000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0


In [47]:
# this is mvp05.calculate_rona with some minor tweaks
def calculate_rona(rona_elements, sig_models, marker_set='all'):
    """Calculate the real rona by summing and averaging elements."""
    print(f'using {marker_set} loci ...')
    
    rona = wrap_defaultdict(None, 3)  # one RONA per pop per climate
    rona_loci_counts = wrap_defaultdict(None, 3)
    all_counts = wrap_defaultdict(list, 2)
    
#     if marker_set != 'all':
#         loci_pkl = op.join(rona_training_dir, f'{seed}_{marker_set}_loci.pkl')  # from MVP_01.py
#         subset_loci = pklload(loci_pkl)
    subset_loci = marker_loci[marker_set]  # added for complex sims
    
    for env, gardendict in rona_elements.items():
        for garden, popdict in pbar(gardendict.items(), desc=env):
            for pop, elementdict in popdict.items():
                # make sure I got what I was expecting to get - number of loci with sig linear models
                assert len(elementdict) == len(sig_models[env])

                # for each group of loci, calc RONA by according to equation in Rellstab et al. (2016) by 
                    # averaging rona_elements while accounting for missing data
                
                # get all of the typical summation elements for loci within `sig_models[env]` that have sig models
                    # these should all be in `elementdict.keys()`
                interloci = set(sig_models[env]).intersection(elementdict.keys())
                assert len(interloci) == len(sig_models[env]) == len(elementdict.keys())
                
                if marker_set != 'all':
                    interloci = set(interloci).intersection(subset_loci)

                elements = [elementdict[locus] for locus in interloci]
                if sum(el==el for el in elements) > 0:
                    # if at least one instance of non-np.nan data:
                    _mean = np.nanmean(elements)  # np.nanmean accounts for missing data
                else:
                    # avoid RuntimeWarning: Mean of empty slice
                    # this happens when a pop has missing data at all of the few number of loci given (eg 2)
                        # and each `el` in `elements` was therefore `np.nan`
                        # this would only happen with empirical data ++ when the loci set is very small
                    _mean = np.nan
                    
                rona[env][garden][pop] = _mean
                rona_loci_counts[env][garden][pop] = len(interloci)
                all_counts[garden][env].append(interloci)

    # save RONA results
#     rona_file = op.join(rona_outdir, f'{seed}_{marker_set}_RONA_results.pkl')
    rona_file = op.join(rona_outdir, f'tutorial_{marker_set}_RONA_results.pkl')  # added for complex sims
    pkldump(rona, rona_file)

    print('\tsaved RONA results to: ', rona_file, '\n')
    
    pass

In [48]:
# calculate RONA according to equation from Rellstab et al. 2016
print(ColorText('\nCalculating RONA for each common garden ...').bold().custom('gold'))
for marker_set in ['all', 'adaptive', 'neutral']:
    calculate_rona(rona_elements, sig_models, marker_set)


Calculating RONA for each common garden ...
using all loci ...


env6_PWM: 100%|███████████████| 100/100 [03:31<00:00,  2.12s/it]


	saved RONA results to:  /home/b.lind/offsets/run_20220919_tutorial/RONA/training/training_outfiles/tutorial_all_RONA_results.pkl 

using adaptive loci ...


env6_PWM: 100%|███████████████| 100/100 [00:17<00:00,  5.88it/s]


	saved RONA results to:  /home/b.lind/offsets/run_20220919_tutorial/RONA/training/training_outfiles/tutorial_adaptive_RONA_results.pkl 

using neutral loci ...


env6_PWM: 100%|███████████████| 100/100 [01:58<00:00,  1.19s/it]


	saved RONA results to:  /home/b.lind/offsets/run_20220919_tutorial/RONA/training/training_outfiles/tutorial_neutral_RONA_results.pkl 



In [49]:
formatclock(dt.now() - t1, exact=True)

'0-01:08:15'